In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import plotly.graph_objects as go
import plotly.express as px
import os

In [47]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [48]:
from sklearn.preprocessing import LabelEncoder

In [49]:
#Indicate columns and target
columns = [
    "id","Airline","Delay","Flight","AirportFrom","AirportTo","DayOfWeek","Time","Length"
]

target = ["Delay"]


In [50]:
file_path = Path("Airlines.csv")
airlines_df = pd.read_csv(file_path)
airlines_df.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [51]:
#Read data into pandas
#data = pd.read_csv("Airlines.csv")

airlines_df = airlines_df.loc[:, columns].copy()
airlines_df

,id,Airline,Delay,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length
0,1,CO,1,269,SFO,IAH,3,15,205
1,2,US,1,1558,PHX,CLT,3,15,222
2,3,AA,1,2400,LAX,DFW,3,20,165
3,4,AA,1,2466,SFO,DFW,3,20,195
4,5,AS,0,108,ANC,SEA,3,30,202
...,...,...,...,...,...,...,...,...,...
539378,539379,CO,0,178,OGG,SNA,5,1439,326
539379,539380,FL,0,398,SEA,ATL,5,1439,305
539380,539381,FL,0,609,SFO,MKE,5,1439,255
539381,539382,UA,1,78,HNL,SFO,5,1439,313


In [52]:
#Drop the null columns where all values are null
airlines_df = airlines_df.dropna(axis='columns', how='all')

# Drop the null rows
airlines_df = airlines_df.dropna()
airlines_df

,id,Airline,Delay,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length
0,1,CO,1,269,SFO,IAH,3,15,205
1,2,US,1,1558,PHX,CLT,3,15,222
2,3,AA,1,2400,LAX,DFW,3,20,165
3,4,AA,1,2466,SFO,DFW,3,20,195
4,5,AS,0,108,ANC,SEA,3,30,202
...,...,...,...,...,...,...,...,...,...
539378,539379,CO,0,178,OGG,SNA,5,1439,326
539379,539380,FL,0,398,SEA,ATL,5,1439,305
539380,539381,FL,0,609,SFO,MKE,5,1439,255
539381,539382,UA,1,78,HNL,SFO,5,1439,313


In [53]:
#Remove info that does not tell us anything about delay or no delay 
#Flight numbers cannot be used as unique identifier b/c some flights have same flight number but
#are based out of different airports
airlines_df = airlines_df.drop(columns =['id','Flight'],axis=1)
airlines_df

,Airline,Delay,AirportFrom,AirportTo,DayOfWeek,Time,Length
0,CO,1,SFO,IAH,3,15,205
1,US,1,PHX,CLT,3,15,222
2,AA,1,LAX,DFW,3,20,165
3,AA,1,SFO,DFW,3,20,195
4,AS,0,ANC,SEA,3,30,202
...,...,...,...,...,...,...,...
539378,CO,0,OGG,SNA,5,1439,326
539379,FL,0,SEA,ATL,5,1439,305
539380,FL,0,SFO,MKE,5,1439,255
539381,UA,1,HNL,SFO,5,1439,313


# Initial Visulizations/Counts Using Plotly

In [54]:
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [55]:
#Create plot grouped by airline to count 
#df = airlines_df[['Airline','Delay']].groupby('Airline').agg('count').reset_index()
#fig = px.bar(airlines_df, x='Airline', y='Delay')
#fig.show()

In [56]:
#Create plot grouped by DayOfWeek to count 
#df = airlines_df[['DayOfWeek','Delay']].groupby('Airline').agg('count').reset_index()
#fig = px.bar(airlines_df, x='DayOfWeek', y='Delay')
#fig.show()

In [57]:

#Create plot grouped by departing airport to count 
#airlines_df = airlines_df[['AirportFrom','Delay']].groupby('AirportFrom').agg('count').reset_index()
#fig = px.bar(airlines_df, x='AirportFrom', y='Delay')
#fig.show()

In [58]:
#Create plot grouped by departing airport to count 
#airlines_df = airlines_df[['AirportTo','Delay']].groupby('AirportTo').agg('count').reset_index()
#fig = px.bar(airlines_df, x='AirportTo', y='Delay')
#fig.show()

In [59]:
#Create plot grouped by departing airport to count 
#airlines_df = data[['Length','Delay']].groupby('Length').agg('count').reset_index()
#fig = px.bar(airlines_df, x='Length', y='Delay')
#fig.show()

# Data Preparation

In [60]:
#Change numbers to day of week to later be converted to 1s and 0s 
days = {1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7:'Sunday'}

In [61]:
airlines_df['DayOfWeek'] = airlines_df['DayOfWeek'].map(days)
airlines_df

,Airline,Delay,AirportFrom,AirportTo,DayOfWeek,Time,Length
0,CO,1,SFO,IAH,Wednesday,15,205
1,US,1,PHX,CLT,Wednesday,15,222
2,AA,1,LAX,DFW,Wednesday,20,165
3,AA,1,SFO,DFW,Wednesday,20,195
4,AS,0,ANC,SEA,Wednesday,30,202
...,...,...,...,...,...,...,...
539378,CO,0,OGG,SNA,Friday,1439,326
539379,FL,0,SEA,ATL,Friday,1439,305
539380,FL,0,SFO,MKE,Friday,1439,255
539381,UA,1,HNL,SFO,Friday,1439,313


In [62]:
# Create our features
# Convert string columns to numbers and drop 'delay' column then assign to X
X = pd.get_dummies(airlines_df, columns=['Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek',]).drop('Delay', axis=1)
X

,Time,Length,Airline_9E,Airline_AA,Airline_AS,Airline_B6,Airline_CO,Airline_DL,Airline_EV,Airline_F9,...,AirportTo_XNA,AirportTo_YAK,AirportTo_YUM,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
0,15,205,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,15,222,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,20,165,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,20,195,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,30,202,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539378,1439,326,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
539379,1439,305,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
539380,1439,255,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
539381,1439,313,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [63]:
X.shape

(539383, 613)

In [64]:
# Create our target
y = airlines_df['Delay']
X.head()

,Time,Length,Airline_9E,Airline_AA,Airline_AS,Airline_B6,Airline_CO,Airline_DL,Airline_EV,Airline_F9,...,AirportTo_XNA,AirportTo_YAK,AirportTo_YUM,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
0,15,205,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,15,222,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,20,165,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,20,195,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,30,202,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [65]:
# Convert the target column values to delayed and not delayed based on their values
#y = {'0': 'no delay'}   
#airlines_df = airlines_df.replace(y)

#airlines_df.reset_index(inplace=True, drop=True)
#airlines_df

In [66]:
X.describe()

,Time,Length,Airline_9E,Airline_AA,Airline_AS,Airline_B6,Airline_CO,Airline_DL,Airline_EV,Airline_F9,...,AirportTo_XNA,AirportTo_YAK,AirportTo_YUM,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
count,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,...,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000
mean,802.728963,132.202007,0.038351,0.084645,0.021267,0.033579,0.039152,0.112981,0.051880,0.011969,...,0.002082,0.000106,0.000627,0.158047,0.134912,0.109303,0.129554,0.169536,0.132262,0.166386
std,278.045911,70.117016,0.192043,0.278353,0.144273,0.180143,0.193957,0.316570,0.221784,0.108747,...,0.045582,0.010279,0.025025,0.364786,0.341629,0.312019,0.335812,0.375225,0.338776,0.372427
min,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,565.000000,81.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,795.000000,115.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1035.000000,162.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1439.000000,655.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [67]:
y.value_counts()

0    299119
1    240264
Name: Delay, dtype: int64

In [68]:
## Labels are the values we want to predict
#labels = np.array(X['Delay'])

# Remove the labels from the features
# axis 1 refers to the columns
#X = X.drop('Delay', axis = 1)
# Saving feature names for later use
#X_list = list(X.columns)
# Convert to numpy array
#X = np.array(X)

In [69]:
#d = {}    
# airlines_df = airlines_df.replace(x)

# Split into training and testing splits

In [70]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [71]:
#Check balances

print(Counter(y_train))
print(Counter(y_test))

Counter({0: 224339, 1: 180198})
Counter({0: 74780, 1: 60066})


# Ensemble Learners
## Used to compare which algorithm results in best performance
## Chosen in an effort to improve the accuracy and robustness of the model, decrease variance of the model, and increase overall performance of the model

# Balanced Random Forest Classifier
## Chosen to rank importance of input variables, run efficiently on large datasets, and are robust against overfitting 

In [72]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_f = BalancedRandomForestClassifier(n_estimators = 100)

In [73]:
random_f = random_f.fit(X_train, y_train, sample_weight=None)

In [74]:
# Calculated the balanced accuracy score
y_pred = random_f.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6064460265144553

In [75]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[45478, 29302],
       [23742, 36324]])

In [76]:
#Create dataframe for the cm 
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,45478,29302
Actual 1,23742,36324


In [77]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, y_pred)

In [78]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,45478,29302
Actual 1,23742,36324


Accuracy Score : 0.6066327514349703
Classification Report
              precision    recall  f1-score   support

           0       0.66      0.61      0.63     74780
           1       0.55      0.60      0.58     60066

    accuracy                           0.61    134846
   macro avg       0.61      0.61      0.60    134846
weighted avg       0.61      0.61      0.61    134846



In [79]:
# Calculate feature importance in the Random Forest model.
importances = random_f.feature_importances_
importances

array([3.08988012e-01, 1.73993797e-01, 1.73509006e-03, 2.76731035e-03,
       1.60856972e-03, 1.68181764e-03, 3.42353841e-03, 3.73013685e-03,
       1.75219766e-03, 1.08732445e-03, 4.12421509e-03, 5.93570959e-04,
       3.17187367e-03, 3.13495364e-03, 2.67754929e-03, 4.28901972e-03,
       3.78345850e-03, 4.47785459e-02, 2.14449543e-03, 4.39353719e-03,
       1.97436893e-04, 6.93505395e-05, 1.05442302e-03, 4.18154140e-06,
       4.98909901e-05, 3.04395132e-05, 1.21611795e-04, 2.73409026e-05,
       6.57277814e-06, 3.42749619e-05, 1.54499960e-04, 1.67343914e-04,
       5.67473822e-04, 2.35724366e-04, 2.79830471e-04, 1.85212188e-04,
       2.59685853e-03, 2.46852954e-04, 1.35262051e-03, 2.62408560e-04,
       1.27377857e-04, 1.17217038e-04, 1.07260613e-03, 6.19654879e-05,
       1.28998630e-04, 5.77462870e-05, 3.25817691e-05, 7.85716717e-04,
       1.93610836e-04, 2.08352047e-04, 3.34768379e-05, 1.53360416e-05,
       2.59209547e-04, 1.44961412e-03, 5.53659345e-04, 2.40502919e-03,
      

In [80]:
# We can sort the features by their importance.
sorted(zip(random_f.feature_importances_, X.columns), reverse=True)

[(0.3089880124705678, 'Time'),
 (0.17399379653673433, 'Length'),
 (0.04477854585424607, 'Airline_WN'),
 (0.02350353774907837, 'DayOfWeek_Thursday'),
 (0.021600997255973284, 'DayOfWeek_Tuesday'),
 (0.020589918413409354, 'DayOfWeek_Sunday'),
 (0.02047438691522934, 'DayOfWeek_Monday'),
 (0.0203090224386119, 'DayOfWeek_Wednesday'),
 (0.01762795607693876, 'DayOfWeek_Friday'),
 (0.01187692509921278, 'DayOfWeek_Saturday'),
 (0.004393537186128096, 'Airline_YV'),
 (0.004289019724323398, 'Airline_UA'),
 (0.0041242150865527065, 'Airline_FL'),
 (0.004027086113549419, 'AirportFrom_ORD'),
 (0.0037834584960950023, 'Airline_US'),
 (0.003730136848504885, 'Airline_DL'),
 (0.0034235384092699515, 'Airline_CO'),
 (0.0031718736744366376, 'Airline_MQ'),
 (0.003134953637343915, 'Airline_OH'),
 (0.0031146578136603646, 'AirportFrom_MDW'),
 (0.002926847500706649, 'AirportTo_DFW'),
 (0.002921600510716934, 'AirportTo_ORD'),
 (0.002870811586665109, 'AirportFrom_LAX'),
 (0.002870801210993355, 'AirportTo_ATL'),
 (0.0

In [81]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.66      0.61      0.60      0.63      0.61      0.37     74780
          1       0.55      0.60      0.61      0.58      0.61      0.37     60066

avg / total       0.61      0.61      0.61      0.61      0.61      0.37    134846

